In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('seaborn-deep')
plt.style.use('fivethirtyeight')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 14
plt.rcParams['figure.figsize'] = (12, 8)

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 400)
import warnings
warnings.filterwarnings('ignore')
import sklearn.base as skb
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
import sklearn.utils as sku
import sklearn.linear_model as sklm
import sklearn.neighbors as skn
import sklearn.ensemble as ske
import catboost as cb
import scipy.stats as sstats
import random
seed = 12
np.random.seed(seed)

from datetime import date

import pandas_profiling as pp

In [2]:
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFECV

In [3]:
df_train = pd.read_csv("hck_train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
# important funtions
def datasetShape(df):
    rows, cols = df.shape
    print("The dataframe has",rows,"rows and",cols,"columns.")
    
# select numerical and categorical features
def divideFeatures(df):
    numerical_features = df.select_dtypes(include=[np.number])
    categorical_features = df.select_dtypes(include=[np.object])
    return numerical_features, categorical_features

In [5]:
# profile = pp.ProfileReport(df_train, title='Pandas Profiling Report', explorative=True)
# profile.to_file("profile.html")

In [6]:
# profile.to_notebook_iframe()

In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11383 entries, 0 to 11382
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              11383 non-null  object 
 1   name                     11383 non-null  object 
 2   age                      11383 non-null  int64  
 3   gender                   11383 non-null  object 
 4   owns_car                 11251 non-null  object 
 5   owns_house               11383 non-null  object 
 6   no_of_children           11193 non-null  float64
 7   net_yearly_income        11383 non-null  float64
 8   no_of_days_employed      11278 non-null  float64
 9   occupation_type          11383 non-null  object 
 10  total_family_members     11352 non-null  float64
 11  migrant_worker           11357 non-null  float64
 12  yearly_debt_payments     11361 non-null  float64
 13  credit_limit             11383 non-null  float64
 14  credit_limit_used(%)  

In [8]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45527 entries, 0 to 45526
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              45527 non-null  object 
 1   name                     45527 non-null  object 
 2   age                      45527 non-null  int64  
 3   gender                   45527 non-null  object 
 4   owns_car                 44980 non-null  object 
 5   owns_house               45527 non-null  object 
 6   no_of_children           44753 non-null  float64
 7   net_yearly_income        45527 non-null  float64
 8   no_of_days_employed      45064 non-null  float64
 9   occupation_type          45527 non-null  object 
 10  total_family_members     45444 non-null  float64
 11  migrant_worker           45440 non-null  float64
 12  yearly_debt_payments     45432 non-null  float64
 13  credit_limit             45527 non-null  float64
 14  credit_limit_used(%)  

In [9]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45527 entries, 0 to 45526
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              45527 non-null  object 
 1   name                     45527 non-null  object 
 2   age                      45527 non-null  int64  
 3   gender                   45527 non-null  object 
 4   owns_car                 44980 non-null  object 
 5   owns_house               45527 non-null  object 
 6   no_of_children           44753 non-null  float64
 7   net_yearly_income        45527 non-null  float64
 8   no_of_days_employed      45064 non-null  float64
 9   occupation_type          45527 non-null  object 
 10  total_family_members     45444 non-null  float64
 11  migrant_worker           45440 non-null  float64
 12  yearly_debt_payments     45432 non-null  float64
 13  credit_limit             45527 non-null  float64
 14  credit_limit_used(%)  

In [10]:
df_train.credit_card_default.values


array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [11]:
y_train = df_train['credit_card_default']
test_id = df_test['customer_id']
all_data = pd.concat([df_train, df_test], axis=0, sort=False)

In [12]:
all_data = all_data.drop(['customer_id'], axis=1)
all_data.head()

,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,ita Bose,46,F,N,Y,0.00,107934.04,612.00,Unknown,1.00,1.00,33070.28,18690.93,73,544.00,2,1,1.00
1,Alper Jonathan,29,M,N,Y,0.00,109862.62,2771.00,Laborers,2.00,0.00,15329.53,37745.19,52,857.00,0,0,0.00
2,Umesh Desai,37,M,N,Y,0.00,230153.17,204.00,Laborers,2.00,0.00,48416.60,41598.36,43,650.00,0,0,0.00
3,Rie,39,F,N,Y,0.00,122325.82,11941.00,Core staff,2.00,0.00,22574.36,32627.76,20,754.00,0,0,0.00
4,McCool,46,M,Y,Y,0.00,387286.00,1459.00,Core staff,1.00,0.00,38282.95,52950.64,75,927.00,0,0,0.00


In [13]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56910 entries, 0 to 11382
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     56910 non-null  object 
 1   age                      56910 non-null  int64  
 2   gender                   56910 non-null  object 
 3   owns_car                 56231 non-null  object 
 4   owns_house               56910 non-null  object 
 5   no_of_children           55946 non-null  float64
 6   net_yearly_income        56910 non-null  float64
 7   no_of_days_employed      56342 non-null  float64
 8   occupation_type          56910 non-null  object 
 9   total_family_members     56796 non-null  float64
 10  migrant_worker           56797 non-null  float64
 11  yearly_debt_payments     56793 non-null  float64
 12  credit_limit             56910 non-null  float64
 13  credit_limit_used(%)     56910 non-null  int64  
 14  credit_score          

In [14]:
all_data['no_of_children'].fillna(round(all_data['no_of_children'].mode()[0]), inplace=True,)



In [15]:
all_data['total_family_members'].fillna(round(all_data['total_family_members'].mode()[0]), inplace=True,)
all_data['migrant_worker'].fillna(round(all_data['migrant_worker'].mode()[0]), inplace=True,)
all_data['yearly_debt_payments'].fillna(round(all_data['yearly_debt_payments'].mean()), inplace=True,)
all_data['credit_score'].fillna(round(all_data['credit_score'].mean()), inplace=True,)
all_data['owns_car'].fillna("False", inplace=True,)
all_data['no_of_days_employed'].fillna(round(all_data['no_of_days_employed'].median()), inplace=True,)

In [16]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56910 entries, 0 to 11382
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     56910 non-null  object 
 1   age                      56910 non-null  int64  
 2   gender                   56910 non-null  object 
 3   owns_car                 56910 non-null  object 
 4   owns_house               56910 non-null  object 
 5   no_of_children           56910 non-null  float64
 6   net_yearly_income        56910 non-null  float64
 7   no_of_days_employed      56910 non-null  float64
 8   occupation_type          56910 non-null  object 
 9   total_family_members     56910 non-null  float64
 10  migrant_worker           56910 non-null  float64
 11  yearly_debt_payments     56910 non-null  float64
 12  credit_limit             56910 non-null  float64
 13  credit_limit_used(%)     56910 non-null  int64  
 14  credit_score          

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [18]:
all_data=MultiColumnLabelEncoder(columns = ['gender','owns_car','owns_house','occupation_type']).fit_transform(all_data)
all_data.head()

,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,ita Bose,46,0,1,1,0.00,107934.04,612.00,17,1.00,1.00,33070.28,18690.93,73,544.00,2,1,1.00
1,Alper Jonathan,29,1,1,1,0.00,109862.62,2771.00,8,2.00,0.00,15329.53,37745.19,52,857.00,0,0,0.00
2,Umesh Desai,37,1,1,1,0.00,230153.17,204.00,8,2.00,0.00,48416.60,41598.36,43,650.00,0,0,0.00
3,Rie,39,0,1,1,0.00,122325.82,11941.00,3,2.00,0.00,22574.36,32627.76,20,754.00,0,0,0.00
4,McCool,46,1,2,1,0.00,387286.00,1459.00,3,1.00,0.00,38282.95,52950.64,75,927.00,0,0,0.00


In [19]:
all_data = all_data.drop(['name'], axis=1)
all_data.head()

,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,46,0,1,1,0.00,107934.04,612.00,17,1.00,1.00,33070.28,18690.93,73,544.00,2,1,1.00
1,29,1,1,1,0.00,109862.62,2771.00,8,2.00,0.00,15329.53,37745.19,52,857.00,0,0,0.00
2,37,1,1,1,0.00,230153.17,204.00,8,2.00,0.00,48416.60,41598.36,43,650.00,0,0,0.00
3,39,0,1,1,0.00,122325.82,11941.00,3,2.00,0.00,22574.36,32627.76,20,754.00,0,0,0.00
4,46,1,2,1,0.00,387286.00,1459.00,3,1.00,0.00,38282.95,52950.64,75,927.00,0,0,0.00


In [20]:
# all_data = all_data.replace({'F': 1, 'M': 2})

In [21]:
all_data.head()

,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,46,0,1,1,0.00,107934.04,612.00,17,1.00,1.00,33070.28,18690.93,73,544.00,2,1,1.00
1,29,1,1,1,0.00,109862.62,2771.00,8,2.00,0.00,15329.53,37745.19,52,857.00,0,0,0.00
2,37,1,1,1,0.00,230153.17,204.00,8,2.00,0.00,48416.60,41598.36,43,650.00,0,0,0.00
3,39,0,1,1,0.00,122325.82,11941.00,3,2.00,0.00,22574.36,32627.76,20,754.00,0,0,0.00
4,46,1,2,1,0.00,387286.00,1459.00,3,1.00,0.00,38282.95,52950.64,75,927.00,0,0,0.00


In [22]:
all_data.describe()

,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
count,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,56910.00,45527.00
mean,39.01,0.34,1.32,0.69,0.41,200266.09,66543.75,10.73,2.16,0.18,31853.68,43432.78,52.14,783.11,0.06,0.05,0.08
std,9.55,0.47,0.49,0.46,0.72,600650.40,138432.42,5.60,0.91,0.38,17281.80,133721.64,29.38,100.58,0.26,0.22,0.27
min,23.00,0.00,0.00,0.00,0.00,27170.61,-3.00,0.00,1.00,0.00,2237.47,3604.65,0.00,500.00,0.00,0.00,0.00
25%,31.00,0.00,1.00,0.00,0.00,126544.77,946.00,6.00,2.00,0.00,19310.88,24004.38,27.00,704.00,0.00,0.00,0.00
50%,39.00,0.00,1.00,1.00,0.00,171972.05,2223.00,10.00,2.00,0.00,29152.35,35720.06,54.00,786.00,0.00,0.00,0.00
75%,47.00,1.00,2.00,1.00,1.00,240479.93,5631.00,17.00,3.00,0.00,40566.14,53465.20,78.00,868.00,0.00,0.00,0.00
max,55.00,2.00,2.00,1.00,11.00,140759012.70,365252.00,18.00,13.00,1.00,328112.86,31129970.49,99.00,949.00,2.00,1.00,1.00


In [23]:
train_dataset =all_data[:len(y_train)]
test_tobepredicted = all_data[len(y_train):]

test_1 = test_tobepredicted.drop(['credit_card_default'],axis = 1)

train_features = train_dataset.copy()
train_labels = train_features.pop('credit_card_default')

In [24]:
X_train, X_test, y_train, y_test = train_test_split(train_features_1,train_labels_1,test_size=0.30)

In [25]:
from sklearn.utils import resample

X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
not_default = X[X.credit_card_default==0]
default = X[X.credit_card_default==1]

# upsample minority
default_upsampled = resample(default,
                          replace=True, # sample with replacement
                          n_samples=len(not_default), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
train_dataset = pd.concat([not_default, default_upsampled])

# check new class counts
train_dataset.credit_card_default.value_counts()

0.00    29308
1.00    29308
Name: credit_card_default, dtype: int64

In [26]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58616 entries, 32114 to 37561
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      58616 non-null  int64  
 1   gender                   58616 non-null  int32  
 2   owns_car                 58616 non-null  int32  
 3   owns_house               58616 non-null  int32  
 4   no_of_children           58616 non-null  float64
 5   net_yearly_income        58616 non-null  float64
 6   no_of_days_employed      58616 non-null  float64
 7   occupation_type          58616 non-null  int32  
 8   total_family_members     58616 non-null  float64
 9   migrant_worker           58616 non-null  float64
 10  yearly_debt_payments     58616 non-null  float64
 11  credit_limit             58616 non-null  float64
 12  credit_limit_used(%)     58616 non-null  int64  
 13  credit_score             58616 non-null  float64
 14  prev_defaults     

In [27]:
train_features = train_dataset.copy()
train_labels = train_features.pop('credit_card_default')

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
import xgboost as xgb

XGB = xgb.XGBClassifier(objective="binary:logistic").fit(X_train, y_train)
xgb_accuracy = XGB.score(X_test, y_test)
xgb_accuracy

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier(verbose=4)
clf_mlp.fit(X_train, y_train)

y_pred_mlp = clf_mlp.predict(X_test)
acc_mlp = accuracy_score(y_test, y_pred_mlp)
print(acc_mlp)

In [ ]:
gb =  GradientBoostingClassifier(learning_rate=0.01)

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(gb, X_train, y_train, scoring=scoring, cv=20)

sorted(scores.keys())
gb_fit_time = scores['fit_time'].mean()
gb_score_time = scores['score_time'].mean()
gb_accuracy = scores['test_accuracy'].mean()
gb_precision = scores['test_precision_macro'].mean()
gb_recall = scores['test_recall_macro'].mean()
gb_f1 = scores['test_f1_weighted'].mean()
gb_roc = scores['test_roc_auc'].mean()

gb_accuracy

In [ ]:
LR = LogisticRegression()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(LR, X_train, y_train, scoring=scoring, cv=5)

sorted(scores.keys())
LR_accuracy = scores['test_accuracy'].mean()

LR_accuracy

In [ ]:
decision_tree = DecisionTreeClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(decision_tree, train_features, train_labels, scoring=scoring, cv=20)

sorted(scores.keys())
dtree_accuracy = scores['test_f1_weighted'].mean()

dtree_accuracy

In [29]:
LDA = LinearDiscriminantAnalysis()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(LDA, train_features, train_labels, scoring=scoring, cv=20)

sorted(scores.keys())
LDA_accuracy = scores['test_f1_weighted'].mean()

LDA_accuracy

0.974380162789662

In [24]:
random_forest = RandomForestClassifier()

scoring = ['accuracy', 'precision_macro', 'recall_macro' , 'f1_weighted', 'roc_auc']
scores = cross_validate(random_forest, train_features, train_labels, scoring=scoring, cv=5)

sorted(scores.keys())
rf_f1 = scores['test_f1_weighted'].mean()


rf_f1

0.9788692799430813

In [25]:
rf_f1_p = scores['test_precision_macro'].mean()
rf_f1_m = scores['test_recall_macro'].mean()
print(rf_f1_m)
print(rf_f1_p)

0.8836981037478313
0.9803301914440784


In [26]:
random_forest = RandomForestClassifier()
random_forest.fit(train_features, train_labels)
prediction = random_forest.predict(test_1)

In [30]:
LDA = LinearDiscriminantAnalysis()
LDA.fit(train_features, train_labels)
prediction = LDA.predict(test_1)

In [ ]:
gb =  GradientBoostingClassifier(learning_rate=0.01)
gb.fit(train_features,train_labels)
prediction = gb.predict(test_1)

In [ ]:
import xgboost as xgb

XGB = xgb.XGBClassifier(objective="binary:logistic").fit(X_train, y_train)
prediction = XGB.predict(test_1)

In [31]:
sub = pd.DataFrame()
sub['customer_id'] = test_id
sub['credit_card_default'] = prediction
sub.to_csv('mysubmission_lda1.csv',index=False)